In [1]:
import pandas as pd
import itertools
import numpy as np
from tqdm import tqdm
import risk

In [2]:
raw1 = pd.read_csv('C:/Users/dblab/Desktop/kimsinhyun/streamlit/ks_ptdrctor_wrtng_info.csv')
raw1

,EFC_YY,QF_GRADE_NM,COUR_NM,USR_NO,WRDN_TOT_GRDE,WRDN_PAS_DIV_NM,FNL_PAS_YN,QF_ITM_NM,ZON_NM
0,2015,2급 장애인스포츠지도사,일반과정,C000014298,55,불합격,N,태권도,경기
1,2015,2급 장애인스포츠지도사,일반과정,C000016602,59,필수과목누락,N,유도,서울
2,2015,2급 장애인스포츠지도사,일반과정,C000019915,64,합격,Y,탁구,경기
3,2015,2급 장애인스포츠지도사,일반과정,C000020112,60,합격,N,사이클,인천
4,2015,2급 장애인스포츠지도사,일반과정,C000024828,68,합격,Y,볼링,경기
...,...,...,...,...,...,...,...,...,...
262864,2021,유소년스포츠지도사,일반과정,P000212025,265,불합격,N,보디빌딩,-
262865,2021,유소년스포츠지도사,일반과정,P000212059,295,불합격,N,당구,-
262866,2021,유소년스포츠지도사,일반과정,P000212309,270,불합격,N,줄넘기,-
262867,2021,유소년스포츠지도사,일반과정,P000212397,320,합격,N,줄넘기,-


In [ ]:
"""
data     : 데이터프레임
start_dim: 시작 차원 수, default = 1
end_dim  : 끝 차원 수,   default = -1
Priority : 속성 재식별 위험도로 우선순위 반환
ex) get_all_combinations(data=raw_data, 1,3)
    -> 원본 데이터 속성들의 1~3차원까지의 속성 조합

return: 속성 조합의 list
"""
#################### reorder columns here to implement priority
def get_all_combinations(data, Priority, start_dim=1,end_dim=-1):
    all_combinations = list()
    if(start_dim == -1):
        start_dim = len(data.columns)
    if(end_dim == -1):
        end_dim = len(data.columns)
    for r in range(start_dim, end_dim+1):
        combinations_object = itertools.combinations(list(data.columns), r)
        combinations_list = list(combinations_object)
        all_combinations += combinations_list
    return list(all_combinations)

"""
모두 같은 값을 가지는 속성(컬럼)들을 순회하면서 drop

return: 드랍된 후의 데이터 프레임 & 어떤 컬림이 드랍됐는지
"""
def is_unique(data):
    dropped_cols = []
    for attr in list(data.columns):
        a = data[attr].to_numpy()
        if (a[0] == a).all():
            data = data.drop(attr,axis=1)
            dropped_cols.append(attr)
    return data, dropped_cols

"""
data: 유일성 검사할 데이터 프레임
comb: 어떤 속성 조합을 검사할 것인지

return: 특정 속성 조합에서 유일한 데이터
"""
def find_unique_data(data, comb):
    #특정 속성조합 comb에서 유일한 데이터 반환
    unique = pd.DataFrame(data[list(comb)].drop_duplicates(keep=False)).reset_index()
    return unique


"""
원본 재식별 위험도 & 원본 재식별도

raw_data:   원본 데이터 프레임
K:          임계치
start_dim:  속성 조합 시작 dimension (get_all_combinations 함수에 설명 있음)
end_dim:    속성 조합 끝 dimension   (get_all_combinations 함수에 설명 있음)
"""
def raw_reidentified_datas(raw_data, K=-1, start_dim=1, end_dim=-1):
    #=============원본 재식별 위험도=============
    single_attr, one_attr, record, table = risk.compute_risk(raw_data.copy())
    Priority = list(one_attr.index)
    st.write(Priority)

    #=============원본 재식별도=============
    #K가 -1이면 전부 검사 (데이터 길이만큼)
    if(K==-1):
        K=len(raw_data)
    
    #모두 같은 값을 가지는 속성 제거
    data,dropped_cols  = is_unique(raw_data)
    print("모두 같은 값을 가져 drop된 속성: ", dropped_cols)
    #속성 조합 반환
    combs = get_all_combinations(data, Priority, start_dim, end_dim)

    reidentified_evidence = pd.DataFrame()
    # loop = tqdm(list(combs), total=len(combs), leave=True)
    loop = stqdm(list(combs))
    for comb in loop:
        #특정 속성 조합에서 유일한 데이터 반환
        raw_unique = find_unique_data(data, comb)
        if(len(raw_unique)!=0):
            reidentified_evidence = pd.concat([reidentified_evidence,raw_unique],axis=0)
            reidentified_evidence = reidentified_evidence.drop_duplicates(subset ="abst_row_num__",keep="first")
        if(len(reidentified_evidence) >= K):
            break
    reidentified_evidence = reidentified_evidence.sort_values("abst_row_num__").reset_index(drop=True)
    return reidentified_evidence

In [8]:
single_attr, one_attr, record, table = risk.compute_risk(raw1.copy())
raw1

,EFC_YY,QF_GRADE_NM,COUR_NM,USR_NO,WRDN_TOT_GRDE,WRDN_PAS_DIV_NM,FNL_PAS_YN,QF_ITM_NM,ZON_NM
0,2015,2급 장애인스포츠지도사,일반과정,C000014298,55,불합격,N,태권도,경기
1,2015,2급 장애인스포츠지도사,일반과정,C000016602,59,필수과목누락,N,유도,서울
2,2015,2급 장애인스포츠지도사,일반과정,C000019915,64,합격,Y,탁구,경기
3,2015,2급 장애인스포츠지도사,일반과정,C000020112,60,합격,N,사이클,인천
4,2015,2급 장애인스포츠지도사,일반과정,C000024828,68,합격,Y,볼링,경기
...,...,...,...,...,...,...,...,...,...
262864,2021,유소년스포츠지도사,일반과정,P000212025,265,불합격,N,보디빌딩,-
262865,2021,유소년스포츠지도사,일반과정,P000212059,295,불합격,N,당구,-
262866,2021,유소년스포츠지도사,일반과정,P000212309,270,불합격,N,줄넘기,-
262867,2021,유소년스포츠지도사,일반과정,P000212397,320,합격,N,줄넘기,-


In [5]:
Priority = list(one_attr.index)
Priority

['USR_NO',
 'WRDN_TOT_GRDE',
 'QF_ITM_NM',
 'ZON_NM',
 'QF_GRADE_NM',
 'EFC_YY',
 'WRDN_PAS_DIV_NM',
 'COUR_NM',
 'FNL_PAS_YN']

In [6]:
reprio = raw1.reindex(columns = Priority)
reprio

,USR_NO,WRDN_TOT_GRDE,QF_ITM_NM,ZON_NM,QF_GRADE_NM,EFC_YY,WRDN_PAS_DIV_NM,COUR_NM,FNL_PAS_YN
0,C000014298,55,태권도,경기,2급 장애인스포츠지도사,2015,불합격,일반과정,N
1,C000016602,59,유도,서울,2급 장애인스포츠지도사,2015,필수과목누락,일반과정,N
2,C000019915,64,탁구,경기,2급 장애인스포츠지도사,2015,합격,일반과정,Y
3,C000020112,60,사이클,인천,2급 장애인스포츠지도사,2015,합격,일반과정,N
4,C000024828,68,볼링,경기,2급 장애인스포츠지도사,2015,합격,일반과정,Y
...,...,...,...,...,...,...,...,...,...
262864,P000212025,265,보디빌딩,-,유소년스포츠지도사,2021,불합격,일반과정,N
262865,P000212059,295,당구,-,유소년스포츠지도사,2021,불합격,일반과정,N
262866,P000212309,270,줄넘기,-,유소년스포츠지도사,2021,불합격,일반과정,N
262867,P000212397,320,줄넘기,-,유소년스포츠지도사,2021,합격,일반과정,N
